In [1]:
import numpy as np
import os
import shutil

def copy_non_zero_segments(source_dir, target_dir):
    """
    Copies NPZ files with non-zero elements from the source directory to the target directory.
    
    Parameters:
        source_dir (str): Path to the directory containing segmented NPZ files.
        target_dir (str): Path to the directory where non-zero NPZ files will be copied.
    """
    # Create the target directory if it does not exist
    os.makedirs(target_dir, exist_ok=True)

    # Iterate through all files in the source directory
    for filename in os.listdir(source_dir):
        # Check if the file is an NPZ file
        if filename.endswith(".npz"):
            source_path = os.path.join(source_dir, filename)
            
            # Load the NPZ file and read the data
            data = np.load(source_path)
            image = data['image']  # Assuming the key is 'image'
            
            # Check if there is any non-zero element
            if np.any(image != 0):
                # Copy the file to the target directory
                target_path = os.path.join(target_dir, filename)
                shutil.copy(source_path, target_path)
                print(f"Copied file: {source_path} to {target_path}")

    print("Copying non-zero segments completed.")

# Example usage
# source_directory = "dataset/segmented_npz/raw_segmented_npz/_8x8/multichannel/winter_wheat/c22_c33/F250/"  # Directory containing segmented NPZ files
# target_directory = "dataset/segmented_npz/processed_segmented_npz/_8x8/multichannel_c22_c33_6/winter_wheat/"  # Directory to save non-zero NPZ files
# copy_non_zero_segments(source_directory, target_directory)

In [2]:
import numpy as np
import os
import shutil
import re

def copy_non_zero_temporal_segments(source_dirs, target_dirs):
    """
    Copies NPZ files with non-zero elements across all days from the source directories to the target directories.
    If all three segments corresponding to a segment number across three days are zero, they are removed.
    
    Parameters:
        source_dirs (list): List of directories containing NPZ files for each day.
        target_dirs (list): List of directories where non-zero NPZ files will be copied.
    """
    assert len(source_dirs) == 3 and len(target_dirs) == 3, "Provide exactly three source and target directories."
    # Extract segment IDs from filenames in each folder
    segment_groups = {}
    for day_idx, source_dir in enumerate(source_dirs):
        for filename in os.listdir(source_dir):
            if filename.endswith(".npz"):
                match = re.search(r'_(\d+)_segment_(\d+)_F250', filename)  # Extract segment ID
                if match:
                    segment_id = match.group(2)
                    if segment_id not in segment_groups:
                        segment_groups[segment_id] = {}
                    segment_groups[segment_id][day_idx] = filename
    
    # Process only segments that exist in all three days
    for segment_id, files in segment_groups.items():
        if len(files) == 3:
            segment_data = []
            
            # Load NPZ files for the segment
            for day_idx in range(3):
                source_path = os.path.join(source_dirs[day_idx], files[day_idx])
                data = np.load(source_path)
                image = data['image']  # Assuming the key is 'image'
                segment_data.append(image)
            
            # Convert to NumPy array for easier checks
            segment_data = np.array(segment_data)  # Shape: (3, height, width, channels)
            
            # Check if all three segments are entirely zero
            if not np.all(segment_data == 0):
                # Copy to target directories
                for day_idx in range(3):
                    os.makedirs(target_dirs[day_idx], exist_ok=True)
                    source_path = os.path.join(source_dirs[day_idx], files[day_idx])
                    target_path = os.path.join(target_dirs[day_idx], files[day_idx])
                    shutil.copy(source_path, target_path)
                    print(f"Copied {files[day_idx]} from {source_path} to {target_path}")
    
    print("Copying non-zero temporal segments completed.")

# Example usage
# source_dirs = [
#     "dataset/segmented_npz/raw_segmented_npz/_8x8/temporal/c22_c33/winter_wheat/F250/_0209/",
#     "dataset/segmented_npz/raw_segmented_npz/_8x8/temporal/c22_c33/winter_wheat/F250/_0912/",
#     "dataset/segmented_npz/raw_segmented_npz/_8x8/temporal/c22_c33/winter_wheat/F250/_1609/"
# ]

# target_dirs = [
#     "dataset/segmented_npz/processed_segmented_npz/_8x8/temporal/c22_c33/winter_wheat/F250/_0209/",
#     "dataset/segmented_npz/processed_segmented_npz/_8x8/temporal/c22_c33/winter_wheat/F250/_0912/",
#     "dataset/segmented_npz/processed_segmented_npz/_8x8/temporal/c22_c33/winter_wheat/F250/_1609/"
# ]

# copy_non_zero_temporal_segments(source_dirs, target_dirs)
